In [1]:
#importing required libraries
import os
import shutil
import time

import folium
import imageio
import webbrowser
import zipfile
import json
import fileinput

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from os import path
from branca.colormap import linear
from selenium import webdriver
from PIL import Image
from pathlib import Path

from glob import glob
import numpy as np
import folium
from folium import plugins
from folium.plugins import HeatMap

In [2]:
#reading 2014 data into respective dataframes
df1 = pd.read_csv('../Data/2014/Divvy_Trips_2014_Q1Q2.csv')
df2 = pd.read_csv('../Data/2014/Divvy_Trips_2014-Q3-07.csv')
df3 = pd.read_csv('../Data/2014/Divvy_Trips_2014-Q3-0809.csv')
df4 = pd.read_csv('../Data/2014/Divvy_Trips_2014-Q4.csv')

In [3]:
#Concatenating the dataframes into a single dataframe
frames = [df1,df2,df3,df4]
dfbike = pd.concat(frames)
dfbike.reset_index(drop=True,inplace=True)
dfbike.head(5)

,trip_id,starttime,stoptime,bikeid,tripduration,from_station_id,from_station_name,to_station_id,to_station_name,usertype,gender,birthyear
0,2355134,6/30/2014 23:57,7/1/2014 0:07,2006,604,131,Lincoln Ave & Belmont Ave,303,Broadway & Cornelia Ave,Subscriber,Male,1988.0
1,2355133,6/30/2014 23:56,7/1/2014 0:00,2217,263,282,Halsted St & Maxwell St,22,May St & Taylor St,Subscriber,Male,1992.0
2,2355130,6/30/2014 23:33,6/30/2014 23:35,2798,126,327,Sheffield Ave & Webster Ave,225,Halsted St & Dickens Ave,Subscriber,Male,1993.0
3,2355129,6/30/2014 23:26,7/1/2014 0:24,173,3481,134,Peoria St & Jackson Blvd,194,State St & Wacker Dr,Subscriber,Female,1988.0
4,2355128,6/30/2014 23:16,6/30/2014 23:26,173,638,320,Loomis St & Lexington St,134,Peoria St & Jackson Blvd,Subscriber,Female,1988.0


In [4]:
dfbike.columns

Index(['trip_id', 'starttime', 'stoptime', 'bikeid', 'tripduration',
       'from_station_id', 'from_station_name', 'to_station_id',
       'to_station_name', 'usertype', 'gender', 'birthyear'],
      dtype='object')

### Saving all the station names in a list "allSt14"

#### Comparing with 2013 data and only adding the stations which are new into a list "added14"
#### Also, checking for the stations that have been in 2013 but not in this year's data and adding it to "removed13"

In [5]:
allSt14 = list(dfbike['from_station_name'].unique())

In [6]:
len(allSt14)

311

#### Saving all the stations from 2013 in a list

In [7]:
allSt13 = ['Michigan Ave & Oak St',
 'Racine Ave & Congress Pkwy',
 'Loomis St & Taylor St',
 'Clark St & Wrightwood Ave',
 'Dearborn St & Adams St',
 'Millennium Park',
 'Clark St & Randolph St',
 'State St & Randolph St',
 'Fairbanks Ct & Grand Ave',
 'Sheffield Ave & Kingsbury St',
 'Michigan Ave & Lake St',
 'Damen Ave & Pierce Ave',
 'Wood St & Milwaukee Ave',
 'Wood St & Division St',
 'Sheffield Ave & Fullerton Ave',
 'Cannon Dr & Fullerton Ave',
 'Jefferson St & Monroe St',
 'Noble St & Milwaukee Ave',
 'Lincoln Ave & Eastwood Ave',
 'Dayton St & North Ave',
 'State St & 16th St',
 'Lake Shore Dr & Monroe St',
 'Wabash Ave & Cermak Rd',
 'McClurg Ct & Illinois St',
 'Clinton St & Washington Blvd',
 'Ashland Ave & Augusta Blvd',
 'Franklin St & Chicago Ave',
 'Canal St & Jackson Blvd',
 'Michigan Ave & Congress Pkwy',
 'Kingsbury St & Erie St',
 'Larrabee St & North Ave',
 'Larrabee St & Kingsbury St',
 'May St & Randolph St',
 'Ogden Ave & Chicago Ave',
 'Adler Planetarium',
 'Lincoln Ave & Fullerton Ave',
 'Franklin St & Lake St',
 'Halsted St & 35th St',
 'Ashland Ave & Chicago Ave',
 'Michigan Ave & Pearson St',
 'Aberdeen St & Jackson Blvd',
 'Larrabee St & Menomonee St',
 'Halsted St & James M Rochford St',
 'Greenwood Ave & 47th St',
 'Wabash Ave & Roosevelt Rd',
 'Pine Grove Ave & Addison St',
 'Franklin St & Jackson Blvd',
 'Clinton St & Lake St',
 'May St & Taylor St',
 'McCormick Place',
 'Clinton St & Madison St',
 'State St & Harrison St',
 'Green St & Milwaukee Ave',
 'Halsted St & Wrightwood Ave',
 'Southport Ave & Wrightwood Ave',
 'Museum Campus',
 'Clinton St & Roosevelt Rd',
 'Lincoln Ave & Armitage Ave',
 'Wells St & Walton St',
 'Daley Center Plaza',
 'State St & Van Buren St',
 'State St & Kinzie St',
 'Desplaines St & Kinzie St',
 'Kimbark Ave & 53rd St',
 'Loomis St & Lexington St',
 'Morgan St & 18th St',
 'Racine Ave & 18th St',
 'Racine Ave & Fullerton Ave',
 'Morgan St & Lake St',
 'Halsted St & Polk St',
 'Montrose Harbor',
 'Sheffield Ave & Willow St',
 'Clinton St & Tilden St',
 'Ashland Ave & Armitage Ave',
 'Clarendon Ave & Leland Ave',
 'Dearborn St & Monroe St',
 'Cottage Grove Ave & 51st St',
 'Calumet Ave & 33rd St',
 'Halsted St & 37th St',
 'Michigan Ave & Washington St',
 'Clark St & Congress Pkwy',
 'Wells St & Erie St',
 'Blackstone Ave & Hyde Park Blvd',
 'Wood St & North Ave',
 'Aberdeen St & Madison St',
 'Wilton Ave & Diversey Pkwy',
 'Carpenter St & Huron St',
 'LaSalle St & Illinois St',
 'Orleans St & Elm St',
 'Clarendon Ave & Gordon Ter',
 'Green St & Randolph St',
 'Eckhart Park',
 'Martin Luther King Dr & Oakwood Blvd',
 'Clark St & Waveland Ave',
 'Ellis Ave & 58th St',
 'Lake Shore Dr & Ohio St',
 'California Ave & 21st St',
 'Greenview Ave & Diversey Pkwy',
 'Lakeview Ave & Fullerton Pkwy',
 'Wood St & Grand Ave',
 'Indiana Ave & Roosevelt Rd',
 'Lincoln Ave & Addison St',
 'Ravenswood Ave & Irving Park Rd',
 'Western Ave & Division St',
 'Broadway & Barry Ave',
 'Clark St & Lincoln Ave',
 'Ashland Ave & Wrightwood Ave',
 'Canal St & Adams St',
 'Columbus Dr & Randolph St',
 'Ashland Ave & Wellington Ave',
 'Greenview Ave & Fullerton Ave',
 'LaSalle St & Washington St',
 'Michigan Ave & 18th St',
 'Canal St & Harrison St',
 'Orleans St & Merchandise Mart Plaza',
 'Sheffield Ave & Wellington Ave',
 'State St & 19th St',
 'Fort Dearborn Dr & 31st St',
 'Indiana Ave & 26th St',
 'Peoria St & Jackson Blvd',
 'Halsted St & 21st St',
 'Southport Ave & Wellington Ave',
 'Blue Island Ave & 18th St',
 'Damen Ave & Wellington Ave',
 'Damen Ave & Coulter St',
 'Clinton St & 18th St',
 'Damen Ave & Augusta Blvd',
 'California Ave & Milwaukee Ave',
 'Damen Ave & Division St',
 'Stave St & Armitage Ave',
 'Western Ave & Winnebago Ave',
 'Campbell Ave & North Ave',
 'Milwaukee Ave & Wabansia Ave',
 'May St & Cullerton St',
 'Claremont Ave & Hirsch St',
 'Loomis St & Jackson Blvd',
 'Ashland Ave & Lake St',
 'Canal St & Madison St',
 'Wabash Ave & 8th St',
 'Congress Pkwy & Ogden Ave',
 'Ogden Ave & Race Ave',
 'Clark St & North Ave',
 'Paulina St & Diversey Pkwy',
 'Southport Ave & Belmont Ave',
 'Lincoln Ave & Diversey Pkwy',
 'Morgan Ave & 14th Pl',
 'Mies van der Rohe Way & Chicago Ave',
 'Clark St & Elm St',
 'Sedgwick St & North Ave',
 'Halsted St & Willow St',
 'State St & Erie St',
 'State St & Pearson St',
 'Dearborn Pkwy & Delaware Pl',
 'Lake Shore Dr & Wellington Ave',
 'Sheffield Ave & Addison St',
 'Sedgwick St & Huron St',
 'Wilton Ave & Belmont Ave',
 'Sangamon St & Washington Blvd',
 'Racine Ave & 13th St',
 'Larrabee St & Webster Ave',
 'Southport Ave & Roscoe St',
 'Wells St & Ohio St',
 'Michigan Ave & 14th St',
 'Damen Ave & Chicago Ave',
 'Streeter Dr & Illinois St',
 'Clybourn Ave & Division St',
 'State St & 35th St',
 'Damen Ave & Cullerton St',
 '900 W Harrison',
 'Wentworth Ave & Archer Ave',
 'Canal St & Monroe St',
 'Sedgwick St & Webster Ave',
 'Wells St & Polk St',
 'Cityfront Plaza & N Water St',
 'State St & Wacker Dr',
 'Wabash Ave & Grand Ave',
 'Ravenswood Ave & Berteau Ave',
 'Michigan Ave & Madison St',
 'Halsted St & Madison St',
 'Sheridan Rd & Irving Park Rd',
 'Theater on the Lake',
 'King Dr & 47th St',
 'Clark St & Wellington Ave',
 'Halsted St & Archer Ave',
 'Prairie Ave & Garfield Blvd',
 'St Clair St & Erie St',
 'Paulina St & 18th St',
 'Damen Ave & Madison St',
 'Indiana Ave & 40th St',
 'Halsted St & 18th St',
 'Emerald Ave & 28th St',
 'Ashland Ave & Division St',
 'May St & Fulton St',
 'Normal Ave & Archer Ave',
 'Western Ave & 21st St',
 'California Ave & Division St',
 'Leavitt St & North Ave',
 'Damen Ave & Grand Ave',
 'Ashland Ave & 21st St',
 'Halsted St & Dickens Ave',
 'Hampden Ct & Diversey Pkwy',
 'Racine Ave & Belmont Ave',
 'Southport Ave & Waveland Ave',
 'Damen Ave & Cortland Ave',
 'Sheridan Rd & Montrose Ave',
 'Damen Ave & Melrose Ave',
 'Lincoln Ave & Roscoe St',
 'Western Ave & Leland Ave',
 'Martin Luther King Dr & 29th St',
 'Clark St & Montrose Ave',
 'Morgan St & Polk St',
 'Clifton Ave & Armitage Ave',
 'Lincoln Ave & Leavitt St',
 'Damen Ave & Leland Ave',
 'Ravenswood Ave & Montrose Ave',
 'Milwaukee Ave & Rockwell St',
 'Wells St & 19th St',
 'Sedgwick St & Schiller St',
 'Woodlawn Ave & 55th St',
 'Shore Drive & 55th St',
 'Clarendon Ave & Junior Ter',
 'Ashland Ave & Belle Plaine Ave',
 'Wolcott Ave & Polk St',
 'State St & 33rd St',
 'State St & 29th St',
 'Hermitage Ave & Polk St',
 'Broadway & Sheridan Rd',
 'Clifton Ave & Lawrence Ave',
 'Wentworth Ave & 24th St',
 'Bissell St & Armitage Ave',
 'Calumet Ave & 18th St',
 'Pine Grove Ave & Irving Park Rd',
 'Logan Blvd & Elston Ave',
 'Lincoln Ave & Waveland Ave',
 'Ashland Ave & Grand Ave',
 'Lake Shore Dr & North Blvd',
 'Cottage Grove Ave & 43rd St',
 'California Ave & North Ave',
 'Racine Ave & 15th St',
 'Western Ave & 24th St',
 'Franklin St & Arcade Pl',
 'Ashland Ave & 13th St',
 'California Ave & Francis Pl',
 'Broadway & Belmont Ave',
 'Halsted St & Maxwell St',
 'Indiana Ave & 31st St',
 'Morgan St & 31st St',
 'Wallace Ave & 35th St',
 'Michigan Ave & Jackson Blvd',
 'Larrabee St & Armitage Ave',
 'LaSalle St & Jackson Blvd',
 'Franklin St & Quincy St',
 'Wells St & Evergreen Ave',
 'Wells St & Concord Ln',
 'Southport Ave & Clark St',
 'Kedzie Ave & Milwaukee Ave',
 'Rhodes Ave & 32nd St',
 'Cottage Grove Ave & Oakwood Blvd',
 'Lake Park Ave & 47th St',
 'Stetson Ave & South Water St',
 'Broadway & Berwyn Ave',
 'Clark St & Schiller St',
 'Halsted St & Roscoe St',
 'Broadway & Argyle St',
 'Paulina St & Montrose Ave',
 'Broadway & Wilson Ave',
 'Broadway & Cornelia Ave',
 'Halsted St & Waveland Ave',
 'Leavitt St & Armitage Ave',
 'Sheffield Ave & Wrightwood Ave',
 'Damen Ave & Charleston St',
 'Leavitt St & Hirsch St',
 'Wood St & Taylor St',
 'Southport Ave & Irving Park Rd',
 'Sheridan Rd & Lawrence Ave',
 'Clark St & Winnemac Ave',
 'Lincoln Ave & Belmont Ave',
 'Sheridan Rd & Buena Ave',
 'Southport Ave & Clybourn Ave',
 'Damen Ave & Sunnyside Ave',
 'Sheffield Ave & Webster Ave',
 'Lake Shore Dr & Belmont Ave',
 'Seeley Ave & Roscoe St',
 'Clark St & Leland Ave',
 'Clark St & Chicago Ave',
 'Ashland Ave & Blackhawk St',
 'Halsted St & Diversey Pkwy',
 'Halsted St & Blackhawk St',
 'Emerald Ave & 31st St',
 'Lake Shore Dr & Diversey Pkwy',
 'Lincoln Ave & Belle Plaine Ave',
 'Stockton Dr & Wrightwood Ave',
 'Kedzie Ave & Palmer Ct',
 'Wolcott Ave & Lawrence Ave',
 'Lake Park Ave & 56th St',
 'Ada St & Washington Blvd',
 'Ashland Ave & Grace St',
 'Calumet Ave & 35th St',
 'Racine Ave & Wrightwood Ave',
 'Cottage Grove Ave & 47th St']

In [8]:
removed13 = list(set(allSt13) - set(allSt14))
len(removed13)

6

In [9]:
added14 = list(set(allSt14) - set(allSt13))
len(added14)

17

# Plotting all added stations of 2014 and removed stations of 2013

In [10]:
#reading the 'final_stations.csv' into a dataframe
#using all the stations co-ordinates and comparing with each of the year's stations added/ removed list and adding locations 
#     to each of the list to plot yearly data.

final = pd.read_csv('final_stations.csv')

In [11]:
final.head()

,Unnamed: 0,Station,Latitude,Longitude
0,0,Ogden Ave & Race Ave,41.891795,-87.658751
1,1,State St & 35th St,41.831036,-87.626798
2,2,Clark St & Elm St,41.902973,-87.631280
3,3,Carpenter St & 63rd St,41.779870,-87.650919
4,4,Leavitt St & Division St,41.902997,-87.683825


In [12]:
#Removing the unnecessary column in the dataframe
final = final.drop('Unnamed: 0', axis =1)

In [13]:
final.head()

,Station,Latitude,Longitude
0,Ogden Ave & Race Ave,41.891795,-87.658751
1,State St & 35th St,41.831036,-87.626798
2,Clark St & Elm St,41.902973,-87.631280
3,Carpenter St & 63rd St,41.779870,-87.650919
4,Leavitt St & Division St,41.902997,-87.683825


In [14]:
#Converting the dataframe into a dictionary
# key : Station name and value: latitude and longitude

locations = dict()

for station in final['Station']:
    locations[station] = {'lat': final[final['Station'] == station]['Latitude'].to_list()[0],
                          'long': final[final['Station'] == station]['Longitude'].to_list()[0]}

#printing out the first item in the dictionary to check the key-value pair
list(locations.items())[0]

('Ogden Ave & Race Ave', {'lat': 41.891795, 'long': -87.658751})

In [15]:
#Adding co-ordinates to all the stations that are added this year
n14 = {k: locations[k] for k in locations.keys() & set(added14)}

#printing out the first item in the dictionary to check the key-value pair
list(n14.items())[0]

('Clark St & Grace St', {'lat': 41.95078, 'long': -87.659172})

In [16]:
#Converting the dictionary into dataframe
st14 = pd.DataFrame.from_dict(n14, orient='index')

In [17]:
#Resetting the index
st14.reset_index(inplace=True)

#Renaming the index column to Station name
st14 = st14.rename(columns = {'index':'Station Name'})

In [18]:
#Adding a column with values of color for every station
st14['color'] = st14.apply(lambda row: 'orange', axis=1)

#printing to check the structure of the dataframe
st14.head(1)

,Station Name,lat,long,color
0,Clark St & Grace St,41.95078,-87.659172,orange


# Location Marker map for interactivity purpose (Station Name)

In [19]:
#setting marker as the color mentioned in the color column of the dataframe
marker = st14.loc[st14['color']!='']

In [20]:
map = folium.Map(location=[marker.lat.mean(), marker.long.mean()], zoom_start=12,
                     width=980,height=590, control_scale=True, tiles='OpenStreetMap')
marker.apply(lambda row:folium.Marker(location=[row["lat"], row["long"]],
        popup=row['Station Name'],
          icon=folium.Icon(color=row['color'])
            ).add_to(map), axis=1)

map

# Circle marker map 2014
## Saving the map for converting into gif

In [21]:
#create a map
m = folium.Map(location=[41.761605, -87.644161], zoom_start=10,
                     width=950,height=700, control_scale=True, tiles='cartodb positron')

title_html = '''
                 <h3 align="left" style="font-size:18px"><b>{}</b></h3>
                 '''.format(' &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp    Year: 2014')   
m.get_root().html.add_child(folium.Element(title_html))

#creates a CircleMarker and adds it to the map
def plotDot(point):
    folium.CircleMarker(location=[point.lat, point.long],             
                        color= "red",
                        radius=3,
                        weight=2
                       ).add_to(m)

#use df.apply(,axis=1) to "iterate" through every row in your dataframe
st14.apply(plotDot, axis = 1)

m

In [22]:
os.chdir('E:\DataVisualization\Assignments\divvy clone 2\cs627-divvy-viswanathduragasi')

In [23]:
m.save('Gif_circlemarker/2014.html')

In [24]:
with fileinput.FileInput('Gif_circlemarker/2014.html', inplace=True) as file:
        for line in file:
            print(line.replace('zoomControl: true', 'zoomControl: false'), end='')

In [25]:
# Convert html files to png (screenshot each html page)

# We use a delay because we dont want to take a screenshot of the browser before the map is loaded
delay=3

os.chdir('E:\DataVisualization\Assignments\divvy clone 2\cs627-divvy-viswanathduragasi\Gif_circlemarker')

fn='2014.html'
tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)

browser = webdriver.Chrome()
browser.get(tmpurl)

#Give the map tiles some time to load
time.sleep(delay)
browser.save_screenshot('ss2014.png')
browser.quit()

In [26]:
image = Image.open('ss2014.png')
box = (0, 0, 1100, 700)
cropped_image = image.crop(box)
cropped_image.save('croppedss2014.png')

# Heatmap 2014
## Saving the map for converting into gif

In [27]:
numlist = st14.reset_index()[['lat', 'long']].values.tolist()


m = folium.Map(location=[41.761605, -87.644161], zoom_start=10,
                     width=950,height=700, control_scale=True, tiles='OpenStreetMap')

title_html = '''
                 <h3 align="left" style="font-size:18px"><b>{}</b></h3>
                 '''.format(' &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp    Year: 2014')   
m.get_root().html.add_child(folium.Element(title_html))


HeatMap(numlist).add_to(folium.FeatureGroup(name='Heat Map').add_to(m))
folium.LayerControl().add_to(m)

m

In [28]:
os.chdir('E:\DataVisualization\Assignments\divvy clone 2\cs627-divvy-viswanathduragasi')

In [29]:
m.save('Gif_heatmap/2014.html')

In [30]:
with fileinput.FileInput('Gif_heatmap/2014.html', inplace=True) as file:
        for line in file:
            print(line.replace('zoomControl: true', 'zoomControl: false'), end='')

In [31]:
# Convert html files to png (screenshot each html page)

# We use a delay because we dont want to take a screenshot of the browser before the map is loaded
delay=3

os.chdir('E:\DataVisualization\Assignments\divvy clone 2\cs627-divvy-viswanathduragasi\Gif_heatmap')

fn='2014.html'
tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)

browser = webdriver.Chrome()
browser.get(tmpurl)

#Give the map tiles some time to load
time.sleep(delay)
browser.save_screenshot('ss2014.png')
browser.quit()

In [32]:
image = Image.open('ss2014.png')
box = (0, 0, 1250, 780)
cropped_image = image.crop(box)
cropped_image.save('croppedss2014.png')